In [36]:
import os
import sys
import pandas as pd
import numpy as np
import torch

In [37]:
from torch.utils.data import Dataset, DataLoader

# Pytorch DataLoader
Pytorch는 데이터를 전처리하고 배치화할 수 있는 클래스를 제공한다.    
`Dataset` 클래스는 데이터를 **전처리**하고 dictionary 또는 list 타입으로 변경할 수 있다.   
`DataLoader` 클래스는 데이터 **1. 셔플 2. 배치화 3. 멀티 프로세스** 기능을 제공한다. 

[OFFICAL DOCUMENT](https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader)

# Table of Contents
- [Dataset](#Dataset)
- [Dataloader](#DataLoader)


## Dataset
- 모든 custom dataset 클래스는 `Dataset()` 클래스를 상속받아야 함.
- `__getitem__()`와 `__len__()` 메소드를 반드시 오버라이딩해야 함. 
- `DataLoader` 클래스가 배치를 만들 때 `Dataset` 인스턴스의 `__getitem__()` 메소드를 사용해 데이터에 접근함
- 해당 Dataset 클래스는 string sequence 데이터를 **tokenize** & **tensorize**한다. 

In [38]:
# !pip install transformers
from torchtext.datasets import AG_NEWS
from transformers import BertTokenizer, BertModel
from typing import Iterator

In [39]:
trainer_iter = AG_NEWS(split = 'train')

In [40]:
try:
    trainer_iter[0]
except NotImplementedError:
    print(f"__getitem__() function not implemented.")

__getitem__() function not implemented.


In [41]:
next(trainer_iter)

(3,
 "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.")

> Dataset Class를 상속받아 Custom_Dataset을 반환하는 과정

In [42]:
class Custom_Dataset(Dataset):

    def __init__(self, data: Iterator):
        self.tokenizer = BertTokenizer.from_pretrained("bert-base-cased")
        self.target = []
        self.text = []
        for target, text in data:
            self.target.append(target)
            self.text.append(text)
  
    def __len__(self):
        return len(self.target)

    def __getitem__(self, index):
        # encode
        token_ids = self.tokenizer.encode(
        text = self.text[index],
        truncation = True,
        padding='max_length'
        )
        
        # tensorize
        return torch.tensor(token_ids), torch.tensor([self.target[index]])




In [43]:
train_dataset = Custom_Dataset(trainer_iter)

In [44]:
train_dataset[0]

(tensor([  101, 19879,  1513, 15218, 27674, 10472, 19417,   113, 11336, 27603,
           114, 11336, 27603,   118,  7219,  5151,  3016, 19879,  1513,  1990,
           117,   165,  1134,  1144,   170,  5244,  1111,  1543,  1218,   118,
         25457,  1105,  5411,   165,  6241,  2399,  1107,  1103,  3948,  2380,
           117,  1144,  4432,  1973,   165,  1157,  7023,  1116,  1113,  1330,
          1226,  1104,  1103,  2319,   119,   102,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,  

In [45]:
len(train_dataset)

119999

In [46]:
# decode to see original text
train_dataset.tokenizer.decode(train_dataset[0][0])

'[CLS] Carlyle Looks Toward Commercial Aerospace ( Reuters ) Reuters - Private investment firm Carlyle Group, \\ which has a reputation for making well - timed and occasionally \\ controversial plays in the defense industry, has quietly placed \\ its bets on another part of the market. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]

In [47]:
# !pip install transformers
from transformers import BertTokenizer, BertModel

In [48]:
tokenizer_bert = BertTokenizer.from_pretrained("klue/bert-base")

## Dataloader

-[DataLoader pytorch official document](https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader)   
-[collate_fn 설명 영문 블로그](https://androidkt.com/create-dataloader-with-collate_fn-for-variable-length-input-in-pytorch/)  

- `dataset`
    - **map-style** dataset
    (`Dataset`)
    - iterable style dataset
      - `__iter__()`
- `batch_size` 
  - int
- `shuffle`
  - bool
- `sampler`
  - data index 이터레이터
  - `Dataset` 및 `DataLoader` 모듈과 같이 별도의 import 필요
  - `RandomSampler(dataset)`: batch 별 random하게 sampling
  - `SequentialSampler(dataset)`: batch 별 sequential하게 sampling

- `collate_fn`
  - List(tuple)의 형태를 입력으로 받는 전처리 함수
  - dataloader에 keyword argument로 함수명을 입력함으로서 적용 가능
  - customizing해서 정의하는 것이 일반적
  - nlp의 경우 (text, label) 형태의 tuple을 입력으로 받아 tokenizing 및 tensorizing 과정을 수행하여 output으로 반환


In [49]:
# def custom_collate_fn(batch):
#   """
#   - batch: list of tuples (input_data(string), target_data(int))
  
#   한 배치 내 문장들을 tokenizing 한 후 텐서로 변환함. 
#   이때, dynamic padding (즉, 같은 배치 내 토큰의 개수가 동일할 수 있도록, 부족한 문장에 [PAD] 토큰을 추가하는 작업)을 적용
#   토큰 개수는 배치 내 가장 긴 문장으로 해야함.
#   또한 최대 길이를 넘는 문장은 최대 길이 이후의 토큰을 제거하도록 해야 함
#   토크나이즈된 결과 값은 텐서 형태로 반환하도록 해야 함
  
#   한 배치 내 레이블(target)은 텐서화 함.
  
#   (input, target) 튜플 형태를 반환.
#   """
#   global tokenizer_bert
  
#   input_list, target_list = zip(*batch) #tuple 별 [0] ,[1] 원소를 unpacking 후 zip으로 할당
  
#   tensorized_input = tokenizer_bert(
#       text=input_list,
#       padding='longest',
#       add_special_tokens=True,
#       truncation=True, #longest는 최대길이를 초과함
#       return_tensors='pt'
#   )
  
#   tensorized_label = torch.tensor(target_list)
  
#   return tensorized_input, tensorized_label
    

In [50]:
from torch.utils.data import RandomSampler, SequentialSampler
train_dataloader = DataLoader(dataset=train_dataset, batch_size = 32, sampler= RandomSampler(train_dataset))#, collate_fn = custom_collate_fn)


In [53]:
# dataloader로부터 batch size 확인
cnt=0
for x, y in train_dataloader:
  print(x.shape, y.shape)
  cnt+=1
  if cnt==10:
    break;

torch.Size([32, 512]) torch.Size([32, 1])
torch.Size([32, 512]) torch.Size([32, 1])
torch.Size([32, 512]) torch.Size([32, 1])
torch.Size([32, 512]) torch.Size([32, 1])
torch.Size([32, 512]) torch.Size([32, 1])
torch.Size([32, 512]) torch.Size([32, 1])
torch.Size([32, 512]) torch.Size([32, 1])
torch.Size([32, 512]) torch.Size([32, 1])
torch.Size([32, 512]) torch.Size([32, 1])
torch.Size([32, 512]) torch.Size([32, 1])
